<a href="https://colab.research.google.com/github/mikodham/intellibots_spring21/blob/main/BaselineCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#REQUIREMENT, #don't forget to turn on gpu
!pip3 install nltk sklearn gensim xgboost keras


In [103]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pprint

#PREPROCESSING-Miko
1. Download Dataset
2. Load Dataset
3. Pre-process sentence into features



###1. Load Dataset

In [8]:
# 1st option, upload on ur gdrive
# from google.colab import files
# files.upload()
# 2nd option, just drag the folder from your folder to drive/content/


# 3rd option, git clone
!pwd 
# !rm -rf intellibots_spring21 # FOR UPDATING DATASET DO NOT RUN THIS LINE
!git clone https://github.com/mikodham/intellibots_spring21/

/content
Cloning into 'intellibots_spring21'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 24 (delta 4), reused 15 (delta 2), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [92]:
DATA_FOLDER = '/content/intellibots_spring21/Dataset/'

def load_data(dataset_name):
    """
    Load data for NN models, with under sampling from Twitter dataset
    :param dataset_name: name of dataset
    :return: X, y
    """
    print('Loading text dataset')
    if dataset_name == 'twitter':
      file_name = DATA_FOLDER + 'sample_data_2.xlsx'
      df = pd.read_excel(file_name)
      df = df.dropna()
      # For balancing #of
      # df_pos = df.loc[df['y'] == 1]
      # df_neg = df.loc[df['y'] == 0]
      # df = pd.concat([df_pos, df_neg.sample(len(df_pos['y']))]) # for balancing
      # df = df.sample(frac=1) # can be used for separating train, test later
      df=df.drop_duplicates() #remove duplicate sentences
      print(df)
      X = df['tweets'].to_numpy()
      y = df['y'].to_numpy()
      return X, y
    elif dataset_name == 'reddit':
      file_name = DATA_FOLDER + 'sample_data_1.xlsx'
      df = pd.read_excel(file_name)
      df = df.dropna()
      df=df.drop_duplicates() #remove duplicate sentences
      print(df)
      x1 = df['title'].to_numpy()
      x2 = df['usertext'].to_numpy()
      y = df['y'].to_numpy()
      return (x1, x2), y
    else:
      raise ValueError("Error: unrecognized dataset")


In [99]:
X, y = load_data(dataset_name='reddit') # "args_lstm.dataset"
# print(X[:5])

Loading text dataset
                            title  ...  y
0           I want to end my life  ...  1
1  Call suicide hotline for help   ...  0
2                I love this song  ...  0
3  I will commit suicide tomorrow  ...  1

[4 rows x 3 columns]
(array(['I want to end my life', 'Call suicide hotline for help ',
       'I love this song', 'I will commit suicide tomorrow'], dtype=object), array(["I'm a loser. I don't want to live.",
       'Don’t give up. Try to make change. Call the hotline if you need help.',
       'It sounds great.', "I'll never wake up the day after tomorrow."],
      dtype=object))


#RNN

#LSTM

In [101]:
def build_LSTM(args):
    inp = Input(shape=(args.max_seq_len, ))
    x = Embedding(input_dim=args.max_num_words, output_dim=args.embedding_dim)(inp)
    x = LSTM(units=args.lstm_units, activation='tanh', dropout=args.dropout_rate, return_sequences=True)(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(args.dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Dense(args.dense_units, activation="relu")(x)
    x = Dropout(args.dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Dense(1, activation="sigmoid")(x)
    
    #Nabila, please modify this part
    model = tf.keras.Sequential([
        # TODO: add a first dense layer with 16 nodes applying ReLU activation function
        tf.keras.layers.Dense(16,activation='relu',input_shape=(NUM_WORDS,)),
        # TODO: add a second dense layer with 16 nodes applying ReLU activation function
        tf.keras.layers.Dense(16, activation='relu'),  
        # TODO: add a last dense layer with only 1 node applying Softmax activation function
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model = Model(inputs=inp, outputs=x)

    model.compile(
        loss='binary_crossentropy', 
        # OR 'sparse_categorical_crossentropy' for multiclassifcation
        optimizer='adam', 
        # OR 'RMSprop'
        metrics=['accuracy'])
    return model
